## 크롤링, 파일에 본문 저장

In [16]:
import urllib.request
from urllib.request import urlopen
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.common import exceptions

class newsCrawler:
    def __init__(self):
        self.titleList=[]
        self.contentsList=[]
        self.imageList=[]
        
    # 네이버 뉴스홈
    def mainCrawl(self):    
        # 정치=100 경제=101 사회=102 생활/문화=103 세계=104 IT/과학=105
        for i in range(100, 102):
            main_url = "https://news.naver.com/main/main.nhn?mode=LSD&mid=shm&sid1="+str(i)
            driver.get(main_url)
            
            # '헤드라인 더보기' 버튼이 있다면 누르기       
            self.showMore()
            driver.implicitly_wait(3)
            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')  
            
            # 헤드라인 가져오기
            self.subCrawl(soup)
        driver.quit()
    
    # 더보기버튼 클릭
    def showMore(self):
        try:
            while True:
                driver.find_element_by_xpath('//*[@id="main_content"]/div/div[2]/div[2]/div/a').click()
        except exceptions.ElementNotVisibleException as e:
            pass
        except Exception as e:
            pass
    
    # 헤드라인 뉴스 크롤링
    def subCrawl(self, soup):
        # 모든 헤드라인 뉴스 저장
        articles = soup.find_all('div', {'class': 'cluster_group _cluster_content'})
        
        for i in range(len(articles)):
            # 각 뉴스 본문에 있는 이미지와 이미지URL를 저장할 리스트
            images=[]
            imagesURL=[]
            
            # 본문 링크
            temp = articles[i].find_all('div', {'class': 'cluster_text'})[0]
            #print(temp.a.text)         
            conURL = temp.a.get('href')
            html2 = session.get(conURL,headers=headers).content
            soup2 = BeautifulSoup(html2, 'html.parser')
            
            # 기사 제목, 내용 추출
            # 기사 요약 부분있으면 본문에서 제외하기
            summary = soup2.find('strong', {'class':'media_end_summary'})
            if summary==None:
                summary=""
            else:
                summary=summary.text
            
            contents = soup2.find('div', id= "articleBodyContents").text.replace("\n"," ").replace(str(summary),"").replace("// flash 오류를 우회하기 위한 함수 추가 function _flash_removeCallback() {}"," ")
            title=soup2.find('h3',id="articleTitle").text
            
            # 이미지 추출
            images=soup2.find_all('span', {'class','end_photo_org'})

            for i in range(len(images)):
                imagesURL.append(images[i].find("img")["src"])
                #print(imagesURL[i])
            
            # 기사 제목과 내용, 이미지URL 각 리스트에 저장
            self.titleList.append(title)
            self.contentsList.append(contents)
            self.imageList.append(imagesURL)
        
        # 파일 저장
        self.saveNewsToFile()
    
    # 파일에 저장(DB에 저장)
    def saveNewsToFile(self):
        f = open('result.txt', 'w', encoding='utf-8')
        for i in range(len(self.titleList)):
            f.write(self.titleList[i] + "\n")
            f.writelines(self.contentsList[i]+"\n")
            f.writelines(self.imageList[i])
            f.writelines("\n\n")
        f.close()

In [17]:
session = requests.Session()

headers = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_5) AppleWebKit 537.36 (KHTML, like Gecko) Chrome",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8"
}

#driver = webdriver.Chrome(r"C:\Users\LCM\Downloads\chromedriver_win32\chromedriver.exe")
driver = webdriver.Chrome(r"C:\Users\seenw\Downloads\chromedriver_win32\chromedriver.exe")

c=newsCrawler()
c.mainCrawl()

## 파일에 요약문 저장

In [12]:
from konlpy.tag import Kkma
from konlpy.tag import Okt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import normalize
import numpy as np


class SentenceTokenizer(object):
    def __init__(self):
        self.kkma = Kkma()
        self.okt = Okt()
        # 불용어 불러오기
        self.stopwords = [line.rstrip('\n') for line in open('stopwords_korean2.txt', encoding = 'utf-8')]
    
    def text2sentences(self, text):
        sentences = self.kkma.sentences(text)\
        
        for idx in range(0, len(sentences)):
            if len(sentences[idx]) <= 10:
                sentences[idx-1] += (' ' + sentences[idx])
                sentences[idx] = ''
                
        return sentences
    
    def get_nouns(self, sentences):
        nouns = []
        for sentence in sentences:
            if sentence != '':
                nouns.append(' '.join([noun for noun in self.okt.nouns(str(sentence))
                    if noun not in self.stopwords and len(noun) > 1]))
                
        return nouns


class GraphMatrix(object):
    def __init__(self):
        self.tfidf = TfidfVectorizer()
        self.cnt_vec = CountVectorizer()
        self.graph_sentence = []
        
    def build_sent_graph(self, sentence):
        tfidf_mat = self.tfidf.fit_transform(sentence).toarray()
        self.graph_sentence = np.dot(tfidf_mat, tfidf_mat.T)
        return self.graph_sentence
    
    def build_words_graph(self, sentence):
        cnt_vec_mat = normalize(self.cnt_vec.fit_transform(sentence).toarray().astype(float), axis=0)
        vocab = self.cnt_vec.vocabulary_
        return np.dot(cnt_vec_mat.T, cnt_vec_mat), {vocab[word] : word for word in vocab}
    
    
class Rank(object):
    def get_ranks(self, graph, d=0.85): # d = damping factor
        A = graph
        matrix_size = A.shape[0]
        for id in range(matrix_size):
            A[id, id] = 0 # diagonal 부분을 0으로
            link_sum = np.sum(A[:,id]) # A[:, id] = A[:][id]
            if link_sum != 0:
                A[:, id] /= link_sum
                
            A[:, id] *= -d
            A[id, id] = 1
            
        B = (1-d) * np.ones((matrix_size, 1))
        ranks = np.linalg.solve(A, B) # 연립방정식 Ax = b
        return {idx: r[0] for idx, r in enumerate(ranks)}

    
    
class TextRank(object):
    def __init__(self, text):
        self.sent_tokenize = SentenceTokenizer()
        self.sentences = self.sent_tokenize.text2sentences(text)
            
        self.nouns = self.sent_tokenize.get_nouns(self.sentences)
        self.graph_matrix = GraphMatrix()
        self.sent_graph = self.graph_matrix.build_sent_graph(self.nouns)
        self.words_graph, self.idx2word = self.graph_matrix.build_words_graph(self.nouns)
        self.rank = Rank()
        self.sent_rank_idx = self.rank.get_ranks(self.sent_graph)
        self.sorted_sent_rank_idx = sorted(self.sent_rank_idx, key=lambda k: self.sent_rank_idx[k], reverse=True)
        self.word_rank_idx = self.rank.get_ranks(self.words_graph)
        self.sorted_word_rank_idx = sorted(self.word_rank_idx, key=lambda k: self.word_rank_idx[k], reverse=True)
        #print(self.nouns)
        
    def summarize(self, sent_num=3):
        summary = []
        index=[]
        for idx in self.sorted_sent_rank_idx[:sent_num]:
            index.append(idx)
        
        index.sort()
        for idx in index:
            summary.append(self.sentences[idx])
        
        return summary
    
    def keywords(self, word_num=10):
        rank = Rank()
        rank_idx = rank.get_ranks(self.words_graph)
        sorted_rank_idx = sorted(rank_idx, key=lambda k: rank_idx[k], reverse=True)
        
        keywords = []
        index=[]
        for idx in sorted_rank_idx[:word_num]:
            index.append(idx)
            
        #index.sort()
        for idx in index:
            keywords.append(self.idx2word[idx])
            
        return keywords

### 문제1. okt 사용자 사전 추가 해야 함. (고유어, 이름 (문재인, 나경원, 오세훈 ), 기타(코스피, 위안부)
### 문제2. 인용문을 한 문장으로 인식하지 않도록 하기.

In [14]:
f = open('result.txt', 'r', encoding='utf-8')

summary = []
count=1

while True:
    line = f.readline()
    if count%4 == 2:
        sum = TextRank(line)
        print(sum.summarize(5))
        print()
#         for row in sum.summarize(7):
#             summary.append(row)
#             print(row)
#         print()
#         print()
        
    if not line:
        break
    
    count+=1

f.close()

['국민의 힘 오 세 훈 서울시장 예비후보가 만약 나 경원 예비후보가 국민의 힘 후보로 정해지면 국민의 당 안 철수 후보와의 최종 단일화가 쉽지 않을 것이라고 주장했습니다.', "오 후보는 오늘 (25 일) 아침 BBS 라디오 ' 박경수의 아침 저널 '에서 나 후보로는 외연 확장이 쉽지 않지만, 안 후보와 자신은 다 열려 있고 반드시 단일화를 이뤄야 한다는 공감대도 있다며 이같이 말했습니다.", '또, 오 후보야말로 강경 보수라는 나 후보의 발언에 대해 선 나 후보가 당원 표가 반영되는 예선에서는 본인이 강경 보수를 자처했었다고 반박했습니다.', '', '부 장원 [boojw1 @ytn .co .kr][ 저작권자 (c) YTN & YTN plus 무단 전재 및 재배포 금지] ▶ 이 시각 코로나 19 확 진자 현황을 확인하세요.']

['연합 뉴스 국민의 힘 오 세 훈 서울시장 예비후보가 25일 토론 평가 단 구성이 당원 중심으로 이뤄 져 있어 평가 결과를 왜곡한다며 토론 평가 단 해체를 주장하고 나섰다.', '오 세 훈 후보는 " 당초 4.7 재· 보궐선거 중앙당 공천관리 위원회는 토론 평가 단 운영을 위해 서울 각 당협별로 해당 선거구 거주자, 공직선거 법상 \' 투표권이 있는 자\', 적극적 자발적 참여 의사가 있는 당원 및 비당원을 대상으로 50 명씩 추천을 받은 후 추첨을 통해 1000명의 토론 평가 단을 구성했다 "며 " 하지만 현행 평가 단은 100% 당 협위 원장의 추천을 받아 구성되었음은 물론 당원 중심의 평가 결과가 시민평가라는 이름으로 언론에 보도됨으로서 심각한 여론 왜곡을 조장하고 있다" 고 주장했다.', '그는 " 실제 이러한 평가 결과에 대해 일반 시민들은 물론 당원들은 공정하지 못하다며 이의를 제기하고 있다" 고 했다.', '이어 " 심지어 어떤 평가 단원은 TV 토론을 시청하지 않았음에도 자동응답 시스템에 응답한 경우도 있다고', '요구 내용은 ▲ 토론 평가 단 해체 ▲ 토론 평가 단의 당원 중심 구성을 밝히고 공개 사과 ▲ 사무총장

['[ 머니 투데이 박광 범 기자] 손 태 승 우리 금융그룹 회장이 25일 오후 서울 여의도 금융감독원에서 열린 사모 펀드 관련 제 7차 제재 심의 위원회에 출석하고 있다.', '금감원은 25일 오후 2시부터 제재 심의 위원회( 제재 심 )를 열고 우리은행과 신한 은행에 대한 부문 검사 결과 조치 안을 심의하고 있다.', '손 태 승 우리 금융 회장은 이날 오후 1시 20분 쯤 제재 심 참석을 위해 금감원에 출석했다.', "앞서 금감원은 라 임 펀드 판매 당시 우리은행장이 었던 손 태 승 우리 금융 지주 회장에 ' 직무정지'( 상당 )를, 진 옥동 신한 은행장은 ' 문책 경고' 의 중징계를 각각 사전 통보 한 상태다.", "여기에 금감원은 라 임 사건과 관련, 신한 금융 지주 차원의 ' 매트릭스 체제 '를 문제 삼아 조 용병 신한 금융 회장에 겐 경징계인 ' 주의적 경고 '를 통보했다."]

['이인 실 ( 사) 한국여성 발명협회 회장 [ 한국여성 발명협회][ 아이 뉴스 24 최 상국 기자] 한국여성 발명협회는 25일 aT 센터에서 개최된 제 22회 정기총회에서 제 11대 회장으로 현 이인 실 회장을 선출, 연임을 확정했다고 발표했다.', '', '이 회장은 이 날 취임사를 통해 “ 여성 발명진흥사업의 포스트 코로나 경쟁력 확보를 위해 대응 역량을 강화” 해 나갈 것을 강조하며 “ 여성 발명 기업인의 경제활동이 활발히 재개되도록 지원” 하겠다고 밝혔다.2019', '', '년 2월부터 제 10대 회장을 역임한 이 회장은 제 11대 회장으로 2023년 2월까지 2년 더 협회를 이끌게 됐다.']

['농심은 오는 3월 비빔 면 신제품 ‘ 배 홍동’ 을 출시한다.', '농심은 오는 3월 비빔 면 신제품 ‘ 배 홍동’ 을 출시한다고 25일 밝혔다.', "농심은 지난해 여름 ' 칼 빔 면' 을 출시하며 비빔 면 시장 공략에 나섰지만 1 위 팔도와 2위 오뚜기 진 비빔 면에 밀려 3위에 그쳤다.", '농심 배 홍동은 배와 홍 고추, 동치미를 갈아 숙성시켜 만든 색다른 비

## 파이썬 데베

In [34]:
import urllib.request
from urllib.request import urlopen
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.common import exceptions
import pymysql

conn = pymysql.connect(host='newdb.c7p2ncpgik7h.ap-northeast-2.rds.amazonaws.com', user='admin', password='1dlckdals!',
                       db='TEST1', charset='utf8')
curs = conn.cursor()
session = requests.Session()
headers = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_5) AppleWebKit 537.36 (KHTML, like Gecko) Chrome",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8"
}
#driver = webdriver.Chrome(r"/home/capston/chromedriver",chrome_options=chrome_options)
#driver = webdriver.Chrome(r"C:\Users\LCM\Downloads\chromedriver_win32\chromedriver.exe")
driver = webdriver.Chrome(r"C:\Users\seenw\Downloads\chromedriver_win32\chromedriver.exe")


class newsCrawler:
    def __init__(self):
        self.titleList=[]
        self.contentsList=[]
        self.imageList=[]
        self.dateList=[]
    # 네이버 뉴스홈
    def mainCrawl(self):    
        # 정치=100 경제=101 사회=102 생활/문화=103 세계=104 IT/과학=105
        for category in range(100, 101):
            main_url = "https://news.naver.com/main/main.nhn?mode=LSD&mid=shm&sid1="+str(category)
            driver.get(main_url)
            
            # '헤드라인 더보기' 버튼이 있다면 누르기       
            self.showMore()
            driver.implicitly_wait(3)
            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')  
            
            # 헤드라인 가져오기
            self.subCrawl(soup,category)
        driver.quit()
    
    # 더보기버튼 클릭
    def showMore(self):
        try:
            while True:
                print("더보기")
                driver.find_element_by_xpath('//*[@id="main_content"]/div/div[2]/div[2]/div/a').click()
                driver.implicitly_wait(0.5)
        except exceptions.ElementNotVisibleException:
            pass
        except Exception:
            pass
    
    

    # 헤드라인 뉴스 크롤링
    def subCrawl(self, soup,category):
        # 모든 헤드라인 뉴스 저장
        articles = soup.find_all('div', {'class': 'cluster_group _cluster_content'})
        
        for i in range(len(articles)):
            # 각 뉴스 본문에 있는 이미지와 이미지URL를 저장할 리스트
            images=[]
            imagesURL=[]

            temp = articles[i].find_all('div', {'class': 'cluster_text'})[0]

            conURL = temp.a.get('href')
            html2 = session.get(conURL,headers=headers).content
            soup2 = BeautifulSoup(html2, 'html.parser')

            summary = soup2.find('strong', {'class':'media_end_summary'})
            if summary==None:
                summary=""
            else:
                summary=summary.text
            
            content = soup2.find('div', id= "articleBodyContents").text.replace("\n"," ").replace(str(summary),"").replace("// flash 오류를 우회하기 위한 함수 추가 function _flash_removeCallback() {}"," ")
            title=soup2.find('h3',id="articleTitle").text
            #print(title)
            date=soup2.find('span', {'class','t11'}).text
            #print(contents)
            #print(date)
            # 이미지 추출
            images=soup2.find_all('span', {'class','end_photo_org'})
            #print(soup2)
            


            for i in range(len(images)):
                imagesURL.append(images[i].find("img")["src"])
                #print(imagesURL[i])
            
            # 기사 제목과 내용, 이미지URL 각 리스트에 저장
            #self.titleList.append(title)
            #self.contentsList.append(contents)
            #self.imageList.append(imagesURL)
            #self.dateList.append(date)
            #print(type(imagesURL[0]))
            #print(title,date,category)
            print(title)
            self.saveToDB(str(title),str(content),str(imagesURL[0]),str(date),str(category))
    

        # DB에 저장
        
        #self.saveToDB(self.titleList,self.contentsList,self.imageList,self.dateList)

    def saveToDB(self,title,content,imagesURL,date,category):
        title = title.replace("'","")
        content=content.replace("'","")

        # SQL문 실행
        sql = "USE TEST1"
        curs.execute(sql)
        '''
        CREATE TABLE NEWS4 (TITLE CHAR(200) NOT NULL,
        CONTENT TEXT NOT NULL,
        DATE VARCHAR(40) NOT NULL,
        CATEGORY VARCHAR(40),
        COUNT int NOT NULL AUTO_INCREMENT,
        IMAGE TEXT NOT NULL,
        CONSTRAINT PLAYER_PK PRIMARY KEY (COUNT));
        '''

        #sql3="insert into NEWS3(title,content,date,category,image) VALUES(" +title+ ',' +content+ ',' +date+ ',' +category+ ',' +imagesURL+ ");"
        sql3="""insert into NEWS4(title,content,date,category,image) VALUES('%s','%s','%s','%s','%s');"""%(title,content,date,category,imagesURL)
        curs.execute(sql3)
        conn.commit()
        print("db updated!")

        return 1


Crawl=newsCrawler()
Crawl.mainCrawl()
conn.commit()
curs.close()
print('done')

더보기


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=88.0.4324.182)
